In [1]:
%config IPCompleter.greedy=True
%matplotlib inline

import csv, json, os, sys
import requests
from time import sleep

import obspy
from obspy import read

from obspy.core.event import read_events
from obspy import read_inventory


DATA_PATH = os.path.expanduser('~/.seisml/mars/')

## Download and Sample
the following:
* net=XB
* sta=ELYSE
* cha=BHU,BHV,BHW
* loc=02

In [9]:
def download_data_availability(data_path=DATA_PATH):
    try:
        os.makedirs(data_path)
    except FileExistsError:
        pass
    
    payload = {
        'option': 'com_ajax',
        'data': 'ELYSE',
        'format': 'json',
        'module': 'seis_data_available'
    }
    r = requests.post('https://www.seis-insight.eu/en/science/seis-data/seis-data-availability', payload)
    with open(os.path.join(data_path, 'data_availability.json'), 'wb') as f:
        f.write(r.content)

In [10]:
download_data_availability()

In [11]:
with open(os.path.join(DATA_PATH, 'data_availability.json'), 'r') as f:
    raw_ava = json.load(f)

In [15]:
ava = []
for t in raw_ava['data']:
    if t['network'] == 'XB' and t['location'] == '02' and t['channel'] == 'BHU':
        ava.append(t)

In [16]:
ava[0]

{'network': 'XB',
 'station': 'ELYSE',
 'location': '02',
 'channel': 'BHU',
 'startTime': '2019-02-12T02:43:01.313000',
 'endTime': '2019-02-12T04:14:37.259000',
 'sampleRate': '20',
 'quality': 'R'}

example data req: `http://ws.ipgp.fr/fdsnws/dataselect/1/query?network=XB&amp;station=ELYSE&amp;startTime=2019-02-12T02:43:01&amp;endTime=2019-02-12T04:14:38&amp;location=02&amp;channel=BH?&amp;nodata=404`

In [18]:
def download_mseed(event, channel='BH?', data_path=DATA_PATH + 'raw/'):
    try:
        os.makedirs(data_path)
    except FileExistsError:
        pass

    payload = {
        'network': event['network'],
        'station': event['station'],
        'startTime': event['startTime'],
        'endTime': event['endTime'],
        'location': event['location'],
        'channel': channel
    }
    
    req = requests.get('http://ws.ipgp.fr/fdsnws/dataselect/1/query', params=payload)
    file_name = '-'.join([event['network'], event['station'], event['location'], event['startTime'], event['endTime']]) + '.mseed'
    print('downloading: %s' % file_name)
    path = os.path.join(data_path, file_name)
    with open(path, 'wb') as c:
        c.write(req.content)
    return path

In [19]:
# download all examples:
for event in ava:
    download_mseed(event)
    sleep(1)

downloading: XB-ELYSE-02-2019-02-12T02:43:01.313000-2019-02-12T04:14:37.259000.mseed
downloading: XB-ELYSE-02-2019-02-12T06:39:54.299000-2019-02-12T07:10:02.247000.mseed
downloading: XB-ELYSE-02-2019-02-15T08:14:08.907000-2019-02-15T08:42:48.854000.mseed
downloading: XB-ELYSE-02-2019-02-19T11:56:36.366000-2019-02-20T12:36:19.190000.mseed
downloading: XB-ELYSE-02-2019-02-25T06:48:00.718000-2019-02-25T06:52:21.668000.mseed
downloading: XB-ELYSE-02-2019-02-28T07:48:10.343000-2019-02-28T07:57:28.292000.mseed
downloading: XB-ELYSE-02-2019-02-28T11:59:55.327000-2019-03-01T04:04:14.200000.mseed
downloading: XB-ELYSE-02-2019-03-01T00:00:04.781000-2019-03-01T00:00:10.231000.mseed
downloading: XB-ELYSE-02-2019-03-01T08:17:33.222000-2019-03-01T14:18:31.170000.mseed
downloading: XB-ELYSE-02-2019-03-01T14:52:03.222000-2019-03-02T05:48:00.129000.mseed
downloading: XB-ELYSE-02-2019-03-02T06:19:20.175000-2019-03-03T05:24:54.997000.mseed
downloading: XB-ELYSE-02-2019-03-03T16:56:00.030000-2019-03-04T14

In [20]:
with open(os.path.join(DATA_PATH + 'raw/', 'catelog.json'), 'w') as f:
    json.dump(ava, f)

In [21]:
file_names = list(filter(lambda f: os.path.splitext(f)[1] == '.mseed', os.listdir(DATA_PATH + 'raw/')))
print('num of files: %i' % len(file_names))

num of files: 145


In [22]:
stream = obspy.read(os.path.join(DATA_PATH + 'raw/', file_names[1]))

In [23]:
stream.filter('highpass', freq=0.1)
stream.resample(sampling_rate=20.0, window='hanning', no_filter=True, strict_length=False)

/home/b/anaconda3/envs/seis/lib/python3.7/site-packages/obspy/signal/headers.py:93: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  ], align=True)


15 Trace(s) in Stream:
XB.ELYSE.02.BHU | 2019-05-01T18:59:49.095000Z - 2019-05-02T01:00:23.045000Z | 20.0 Hz, 432680 samples
XB.ELYSE.02.BHU | 2019-05-02T00:59:31.054000Z - 2019-05-02T07:10:38.004000Z | 20.0 Hz, 445340 samples
XB.ELYSE.02.BHU | 2019-05-02T07:09:46.034000Z - 2019-05-02T08:00:13.984000Z | 20.0 Hz, 60560 samples
XB.ELYSE.02.BHU | 2019-05-02T07:59:22.031000Z - 2019-05-02T14:00:13.981000Z | 20.0 Hz, 433040 samples
XB.ELYSE.02.BHU | 2019-05-02T13:59:21.992000Z - 2019-05-02T19:00:13.942000Z | 20.0 Hz, 361040 samples
XB.ELYSE.02.BHV | 2019-05-01T18:59:49.095000Z - 2019-05-02T01:00:23.045000Z | 20.0 Hz, 432680 samples
XB.ELYSE.02.BHV | 2019-05-02T00:59:31.054000Z - 2019-05-02T07:10:38.004000Z | 20.0 Hz, 445340 samples
XB.ELYSE.02.BHV | 2019-05-02T07:09:46.034000Z - 2019-05-02T08:00:13.984000Z | 20.0 Hz, 60560 samples
XB.ELYSE.02.BHV | 2019-05-02T07:59:22.031000Z - 2019-05-02T14:00:13.981000Z | 20.0 Hz, 433040 samples
XB.ELYSE.02.BHV | 2019-05-02T13:59:21.992000Z - 2019-05-02T19

In [24]:
stream[0].meta

         network: XB
         station: ELYSE
        location: 02
         channel: BHU
       starttime: 2019-05-01T18:59:49.095000Z
         endtime: 2019-05-02T01:00:23.045000Z
   sampling_rate: 20.0
           delta: 0.05
            npts: 432680
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'dataquality': 'R', 'number_of_records': 1707, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 512, 'filesize': 8547328})
      processing: ["ObsPy 1.1.0: filter(options={'freq': 0.1}::type='highpass')", "ObsPy 1.1.0: resample(no_filter=True::sampling_rate=20.0::strict_length=False::window='hanning')"]

## Visualize data around recorded marsquakes

In [25]:
event_times = [
    ('2019-12-20T13:12:36', '2019-12-20T13:27:56', '2.4Hz-B') # 2.4Hz -- B
]

non_event_times = [
    
]

In [3]:
event_example_dir = DATA_PATH + 'event_examples'

In [34]:
for t in event_times:
    event = {
        'network': 'XB',
        'station': 'ELYSE',
        'startTime': t[0],
        'endTime': t[1],
        'location': '02'
    }
    download_mseed(event, channel='BH?', data_path=event_example_dir)
    sleep(1)

downloading: XB-ELYSE-02-2019-12-20T13:12:36-2019-12-20T13:27:56.mseed


In [8]:
event_file_names = list(filter(lambda f: os.path.splitext(f)[1] == '.mseed', os.listdir(event_example_dir)))

In [9]:
stream = obspy.read(os.path.join(event_example_dir, event_file_names[1]))
stream.filter('bandpass', freqmin=0.1, freqmax=9.0)

/home/b/anaconda3/envs/seis/lib/python3.7/site-packages/obspy/signal/headers.py:93: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  ], align=True)


3 Trace(s) in Stream:
XB.ELYSE.02.BHU | 2019-12-20T13:12:24.187000Z - 2019-12-20T13:28:02.737000Z | 20.0 Hz, 18772 samples
XB.ELYSE.02.BHV | 2019-12-20T13:12:22.837000Z - 2019-12-20T13:28:14.737000Z | 20.0 Hz, 19039 samples
XB.ELYSE.02.BHW | 2019-12-20T13:12:20.337000Z - 2019-12-20T13:28:15.337000Z | 20.0 Hz, 19101 samples

In [55]:
events = read_events(os.path.join(DATA_PATH, 'events_extended_mars_preferredorigin_2020-01-01.xml'))

In [56]:
for pick in events[0].picks:
    print(pick.phase_hint + ':' + str(pick.time))

P_spectral_end:2019-12-31T17:46:12.696898Z
P_spectral_start:2019-12-31T17:33:53.449867Z
Peak_M2.4:2019-12-31T17:39:21.130288Z
noise_end:2019-12-31T17:29:31.305531Z
noise_start:2019-12-31T17:23:19.060572Z
end:2019-12-31T17:48:39.497726Z
start:2019-12-31T17:33:14.128216Z


In [57]:
events[10].picks

[Pick
	     resource_id: ResourceIdentifier(id="smi:insight.mqs/Pick/20200107105246.21803.66690")
	            time: UTCDateTime(2019, 12, 26, 15, 6, 45, 837740)
	     waveform_id: WaveformStreamID(network_code='XB', station_code='ELYSE', channel_code='BHN', location_code='02')
	       filter_id: ResourceIdentifier(id="smi:local/sc3filter/BW(6,2.2,2.8)__STALTA(100,1000)")
	      phase_hint: 'end'
	 evaluation_mode: 'manual'
	   creation_info: CreationInfo(agency_id='mqs', author='sc', creation_time=UTCDateTime(2020, 1, 7, 10, 52, 46, 218144)),
 Pick
	     resource_id: ResourceIdentifier(id="smi:insight.mqs/Pick/20191227170730.34993.231138")
	            time: UTCDateTime(2019, 12, 26, 14, 51, 10, 926410)
	     waveform_id: WaveformStreamID(network_code='XB', station_code='ELYSE', channel_code='BHZ', location_code='02')
	       filter_id: ResourceIdentifier(id="smi:local/sc3filter/BW_HP(3,0.001)")
	      phase_hint: 'start'
	 evaluation_mode: 'manual'
	   creation_info: CreationInfo(age

In [63]:
event_samples = []

for event in events:
    name = None
    
    for des in event.event_descriptions:
        if des.type == 'earthquake name':
            name = des.text
    
    for pick in event.picks:
        if pick.phase_hint == 'start':
            start = pick.time
#             end = pick.time + 10
#             event_samples.append((name + '_P_spectral_start', start, end))
        elif pick.phase_hint == 'end':
#             start = pick.time - 10
            end = pick.time
    event_samples.append((name, start, end)) 
        

In [64]:
event_samples

[('S0389a', 2019-12-31T17:33:14.128216Z, 2019-12-31T17:48:39.497726Z),
 ('S0388b', 2019-12-30T18:25:38.790191Z, 2019-12-30T18:39:19.308471Z),
 ('S0388a', 2019-12-30T15:49:50.063979Z, 2019-12-30T16:02:04.354336Z),
 ('S0387a', 2019-12-29T14:40:41.228470Z, 2019-12-29T14:57:01.143483Z),
 ('S0386b', 2019-12-28T18:05:56.377063Z, 2019-12-28T18:19:51.518131Z),
 ('S0386a', 2019-12-28T14:54:47.978301Z, 2019-12-28T15:11:07.953657Z),
 ('S0385c', 2019-12-27T15:09:23.651693Z, 2019-12-27T15:25:00.536506Z),
 ('S0385b', 2019-12-27T13:57:17.474737Z, 2019-12-27T14:08:10.550263Z),
 ('S0385a', 2019-12-27T12:41:08.608345Z, 2019-12-27T12:58:43.634679Z),
 ('S0384d', 2019-12-26T16:45:04.755575Z, 2019-12-26T17:01:01.899382Z),
 ('S0384c', 2019-12-26T14:51:10.926410Z, 2019-12-26T15:06:45.837740Z),
 ('S0384b', 2019-12-26T13:09:43.134175Z, 2019-12-26T13:27:26.158499Z),
 ('S0384a', 2019-12-26T11:55:04.175727Z, 2019-12-26T12:10:29.656604Z),
 ('S0383c', 2019-12-25T15:45:17.228113Z, 2019-12-25T16:01:54.132142Z),
 ('S03

In [77]:
for sample in event_samples:
    event = {
        'network': 'XB',
        'station': 'ELYSE',
        'startTime': str(sample[1]),
        'endTime': str(sample[2]),
        'location': '02'
    }
    path = download_mseed(event, channel='BH?', data_path=DATA_PATH + 'events/')
    print(path)
    try:
        stream  = obspy.read(path)
    except: 
        print('cannot load stream')
        continue
    
    stream.plot(outfile = os.path.join(DATA_PATH + 'event_images/', sample[0] + '_raw.png'))
    stream = stream.detrend('linear')
    stream = stream.filter('bandpass', freqmin=1.0, freqmax=5.0)
    stream.plot(outfile = os.path.join(DATA_PATH + 'event_images/', sample[0] + '.png'))
    sleep(1)

downloading: XB-ELYSE-02-2019-12-31T17:33:14.128216Z-2019-12-31T17:48:39.497726Z.mseed
/home/b/.seisml/mars/events/XB-ELYSE-02-2019-12-31T17:33:14.128216Z-2019-12-31T17:48:39.497726Z.mseed
downloading: XB-ELYSE-02-2019-12-30T18:25:38.790191Z-2019-12-30T18:39:19.308471Z.mseed
/home/b/.seisml/mars/events/XB-ELYSE-02-2019-12-30T18:25:38.790191Z-2019-12-30T18:39:19.308471Z.mseed
downloading: XB-ELYSE-02-2019-12-30T15:49:50.063979Z-2019-12-30T16:02:04.354336Z.mseed
/home/b/.seisml/mars/events/XB-ELYSE-02-2019-12-30T15:49:50.063979Z-2019-12-30T16:02:04.354336Z.mseed
downloading: XB-ELYSE-02-2019-12-29T14:40:41.228470Z-2019-12-29T14:57:01.143483Z.mseed
/home/b/.seisml/mars/events/XB-ELYSE-02-2019-12-29T14:40:41.228470Z-2019-12-29T14:57:01.143483Z.mseed
downloading: XB-ELYSE-02-2019-12-28T18:05:56.377063Z-2019-12-28T18:19:51.518131Z.mseed
/home/b/.seisml/mars/events/XB-ELYSE-02-2019-12-28T18:05:56.377063Z-2019-12-28T18:19:51.518131Z.mseed
downloading: XB-ELYSE-02-2019-12-28T14:54:47.978301Z-20

In [60]:
str(event_samples[0][1])

'2019-12-31T17:33:14.128216Z'